<a href="https://colab.research.google.com/github/yu072333/114-/blob/main/%E7%A8%8B%E8%A8%AD%EF%BC%A8%EF%BC%B7%EF%BC%95.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
import pandas as pd
import gradio as gr
import random
import folium
from geopy.geocoders import Nominatim
from geopy.distance import geodesic
import time

# 初始化地理編碼服務
geolocator = Nominatim(user_agent="date_spot_recommender_v1")

def get_coordinates(location_name):
    try:
        # 增加 "Taiwan" 確保搜尋範圍在台灣為主
        location = geolocator.geocode(f"{location_name}, Taiwan", timeout=10)
        if location:
            return (location.latitude, location.longitude)
        return None
    except Exception as e:
        print("geocode error:", e)
        return None

# 解析地名：嘗試從各種標籤中找到可讀的名稱
def resolve_name_from_tags(tags):
    for key in ("name", "alt_name", "official_name", "short_name", "brand", "description"):
        v = tags.get(key)
        if v:
            return v
    return None

# 反向地理編碼 (用來補救沒有名字的地點，但限制次數以免被鎖)
def reverse_name(lat, lon):
    try:
        time.sleep(1)
        r = geolocator.reverse((lat, lon), exactly_one=True, timeout=10)
        if r and r.address:
            return r.address.split(",")[0]
    except Exception as e:
        print("reverse geocode error:", e)
    return None

def fetch_attractions(lat, lon, radius=2000, max_reverse=6):
    """
    從 OSM 抓取資料。
    修改點：增加了 amenity (cafe, bar, restaurant, cinema) 以符合約會需求。
    """
    # 擴大查詢範圍，包含餐飲與娛樂
    query = f"""
    [out:json][timeout:25];
    (
      node["tourism"](around:{radius},{lat},{lon});
      way["tourism"](around:{radius},{lat},{lon});
      node["leisure"](around:{radius},{lat},{lon});
      way["leisure"](around:{radius},{lat},{lon});
      node["amenity"~"cafe|bar|restaurant|cinema"](around:{radius},{lat},{lon});
      way["amenity"~"cafe|bar|restaurant|cinema"](around:{radius},{lat},{lon});
    );
    out center;
    """
    url = "https://overpass-api.de/api/interpreter"
    try:
        resp = requests.post(url, data={'data': query}, timeout=40)
        resp.raise_for_status()
        data = resp.json()
    except Exception as e:
        print("Overpass error:", e)
        return pd.DataFrame()

    items = []
    reverse_calls = 0

    # 定義約會專用的隨機描述庫
    romantic_descriptions = [
        '燈光美氣氛佳', '適合情侶低語', '百萬夜景第一排', '隱密性高',
        '適合告白', '拍照超夢幻', '文青情侶必去', '老派浪漫約會地',
        '安靜舒適', '可以待一整個下午'
    ]

    for el in data.get("elements", []):
        tags = el.get("tags", {}) or {}

        # 取得座標
        if el.get("type") in ("way", "relation"):
            cent = el.get("center") or {}
            lat_el = cent.get("lat")
            lon_el = cent.get("lon")
        else:
            lat_el = el.get("lat")
            lon_el = el.get("lon")

        if lat_el is None or lon_el is None:
            continue

        name = resolve_name_from_tags(tags)

        # 如果沒有名字，嘗試反查 (限量)
        if not name:
            if reverse_calls < max_reverse:
                rname = reverse_name(lat_el, lon_el)
                reverse_calls += 1
                if rname:
                    name = rname

        if not name:
            continue # 如果真的沒名字，約會地點通常不推薦去"未知地點"，直接跳過

        # 判斷類型
        typ = tags.get("tourism") or tags.get("leisure") or tags.get("amenity") or "一般景點"

        # 計算距離
        try:
            dist_m = geodesic((lat, lon), (lat_el, lon_el)).meters
        except Exception:
            dist_m = None

        if dist_m is None:
            dist_bucket = "未知"
        elif dist_m <= 500:
            dist_bucket = "走路可到 (近)"
        elif dist_m <= 1500:
            dist_bucket = "散步一段 (中)"
        else:
            dist_bucket = "建議騎車/開車 (遠)"

        # 模擬約會屬性 (因為 OSM 沒有這些標籤，故用隨機模擬演示功能)
        items.append({
            "地點名稱": name,
            "類型": typ,
            "地址": tags.get("addr:street", "詳見地圖"),
            "浪漫指數": random.choice(['⭐⭐⭐⭐⭐', '⭐⭐⭐⭐', '⭐⭐⭐']),
            "消費門檻": random.choice(['免費/低消', '中等價位', '高價位']),
            "距離_m": int(dist_m) if dist_m is not None else 9999,
            "距離": dist_bucket,
            "適合放閃": random.choice(['是', '否']),
            "約會亮點": tags.get("description") or random.choice(romantic_descriptions),
            "經度": lon_el,
            "緯度": lat_el
        })

    df = pd.DataFrame(items)
    if not df.empty:
        df = df.drop_duplicates(subset=["地點名稱", "經度", "緯度"]).reset_index(drop=True)
    print(f"✅ 找到 {len(df)} 個潛在約會地點")
    return df

def safe_list(xs):
    if xs is None:
        return []
    return list(xs)

def filter_and_recommend(df, type_filter, distance_filter, budget_friendly, photo_friendly):
    if df is None or df.empty:
        return pd.DataFrame(), "這裡似乎是約會沙漠，找不到地點。", ""

    # 處理篩選條件
    type_list = safe_list(type_filter)
    # 如果沒選類型，針對「所有類型」進行對應，避免全空
    if len(type_list) == 0:
        # 這裡做一個簡單的包含檢查，只要類型字串裡有 filter 的關鍵字即可
        pass

    distance_list = safe_list(distance_filter)

    # 建立遮罩
    mask = pd.Series([True] * len(df))

    # 類型篩選 (Keyword matching)
    if len(type_list) > 0:
        # 因為 OSM tags 很雜，我們用字串包含來篩選
        # 例如選 'cafe'，只要類型欄位包含 'cafe' 就算符合
        type_mask = pd.Series([False] * len(df))
        for t in type_list:
            type_mask = type_mask | df['類型'].astype(str).str.contains(t, case=False, regex=False)
        mask = mask & type_mask

    # 距離篩選
    if len(distance_list) > 0:
        mask = mask & df['距離'].isin(distance_list)

    # 預算篩選
    if budget_friendly:
        mask = mask & (df['消費門檻'] == '免費/低消')

    # 拍照篩選
    if photo_friendly:
        mask = mask & (df['適合放閃'] == '是')

    filtered = df[mask].copy()

    # 如果篩選後沒東西，放寬條件 (Fallback)
    if filtered.empty:
        return pd.DataFrame(), "找不到完全符合條件的地點，試著放寬條件（例如：不限制預算）？", ""

    # 隨機推薦 3 個
    n_recommend = min(3, len(filtered))
    recommended = filtered.sample(n=n_recommend, replace=False)

    # 產生文字清單
    recommend_list = "\n".join([
        f"💖 {i+1}. {row['地點名稱']}\n   - 類型：{row['類型']} | 距離：{row['距離']}\n   - 亮點：{row['約會亮點']} ({row['浪漫指數']})"
        for i, row in recommended.iterrows()
    ])

    # 產生給 AI summary 的原始字串
    ai_input = "、".join([
        f"{row['地點名稱']} ({row['類型']}，{row['約會亮點']})"
        for _, row in recommended.iterrows()
    ])

    return recommended, recommend_list, ai_input

def generate_date_plan(ai_input):
    if not ai_input:
        return "請先搜尋以產生約會建議。"
    parts = ai_input.split("、")

    plan = f"🌹 **AI 戀愛軍師建議行程**：\n"
    if len(parts) >= 2:
        plan += f"1. 先去 **{parts[0]}** 走走，{parts[0].split('，')[-1]}。\n"
        plan += f"2. 接著前往 **{parts[1]}**，享受兩人世界。\n"
        plan += "✨ 小撇步：記得根據天氣準備雨具或外套，展現貼心的一面！"
    else:
        plan += f"推薦專注行程：前往 **{parts[0]}**，那裡 {parts[0].split('，')[-1]}，非常適合深度交流。"

    return plan

def search_and_recommend(location_name, type_filter, distance_filter, budget_friendly, photo_friendly, radius_km=2):
    if not location_name or str(location_name).strip()=="":
        return pd.DataFrame(), "請輸入約會地點（例：中山站）", "", "沒有地圖"

    coords = get_coordinates(location_name)
    if not coords:
        return pd.DataFrame(), "找不到這個地點，請換個說法（例如：台北101）", "", "沒有地圖"

    lat, lon = coords
    df = fetch_attractions(lat, lon, radius=int(radius_km*1000))

    recommended_df, recommend_txt, ai_input = filter_and_recommend(df, type_filter, distance_filter, budget_friendly, photo_friendly)
    ai_summary = generate_date_plan(ai_input)

    # 繪製地圖
    map_obj = folium.Map(location=[lat, lon], zoom_start=15)

    # 中心點
    folium.Marker(
        [lat, lon],
        popup="約會集合點",
        icon=folium.Icon(color="gray", icon="user")
    ).add_to(map_obj)

    if not recommended_df.empty:
        for _, row in recommended_df.iterrows():
            # 使用紅色愛心標記
            folium.Marker(
                [row["緯度"], row["經度"]],
                popup=f"<b>{row['地點名稱']}</b><br>{row['約會亮點']}",
                tooltip=f"{row['地點名稱']} ({row['浪漫指數']})",
                icon=folium.Icon(color="red", icon="heart", prefix='fa')
            ).add_to(map_obj)

    # 整理輸出表格
    if not recommended_df.empty:
        out_table = recommended_df[['地點名稱', '類型', '約會亮點', '消費門檻', '距離']].reset_index(drop=True)
    else:
        out_table = pd.DataFrame(columns=['地點名稱', '類型', '約會亮點', '消費門檻', '距離'])

    return out_table, recommend_txt, ai_summary, map_obj._repr_html_()

# ---------------- Gradio UI 設定 ----------------

custom_css = """
#title {text-align: center; color: #E91E63;}
"""

with gr.Blocks(title="情侶約會行程推薦神器", css=custom_css) as demo:
    gr.Markdown("# 👩‍❤️‍👨 情侶約會行程推薦神器\n輸入見面地點，幫你規劃完美的約會行程！")

    with gr.Row():
        with gr.Column(scale=1):
            location_input = gr.Textbox(label="📍 約會集合地點（例：西門町、淡水捷運站）", value="中山捷運站")

            # 對應 OSM 的 tags
            type_options = [
                ("☕ 咖啡廳/下午茶", "cafe"),
                ("🍴 餐廳/美食", "restaurant"),
                ("🍸 酒吧/微醺", "bar"),
                ("🌄 看風景/夜景", "viewpoint"),
                ("🌳 公園散步", "park"),
                ("🎨 美術館/文創", "museum"),
                ("🎬 電影院", "cinema")
            ]

            type_select = gr.CheckboxGroup(
                choices=[t[1] for t in type_options],
                label="💖 想要去哪種地方？（多選）",
                # 顯示標籤名稱，回傳 value
                type="value",
                value=["cafe", "viewpoint", "park"]
            )
            # 為了讓 UI 顯示中文選項，我們需要一點小技巧或直接用中文當 value
            # 簡化起見，這裡直接讓 user 選 raw tag value，但在 label 標註中文

            distance_select = gr.CheckboxGroup(
                ['走路可到 (近)', '散步一段 (中)', '建議騎車/開車 (遠)'],
                label="🚶 移動距離偏好",
                value=['走路可到 (近)', '散步一段 (中)']
            )

            with gr.Row():
                budget_toggle = gr.Checkbox(label="💰 小資約會 (免費/低消)", value=False)
                photo_toggle = gr.Checkbox(label="📸 適合放閃拍照", value=True)

            search_btn = gr.Button("💘 開始規劃約會", variant="primary")

        with gr.Column(scale=2):
            output_map = gr.HTML(label="約會地圖")
            output_summary = gr.Textbox(label="💌 AI 戀愛軍師建議")
            output_text = gr.Textbox(label="精選地點詳情")
            output_table = gr.DataFrame(visible=False) # 表格隱藏，用文字顯示較有溫度，或視需求開啟

    def main_wrapper(location, type_, distance, budget, photo):
        try:
            table, text, summary, map_html = search_and_recommend(location, type_, distance, budget, photo)
            return map_html, summary, text, table
        except Exception as e:
            print("wrapper error:", e)
            return "沒有地圖", "系統發生錯誤，請稍後再試。", str(e), pd.DataFrame()

    search_btn.click(
        fn=main_wrapper,
        inputs=[location_input, type_select, distance_select, budget_toggle, photo_toggle],
        outputs=[output_map, output_summary, output_text, output_table]
    )

if __name__ == "__main__":
    demo.launch(inbrowser=True)

/tmp/ipython-input-508529731.py:276: DeprecationWarning: The 'css' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'css' to Blocks.launch() instead.
  with gr.Blocks(title="情侶約會行程推薦神器", css=custom_css) as demo:


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://25a6f67d383f6abb7d.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
